In [36]:
from collections import defaultdict
import pandas as pd
from surprise import Reader, Dataset
from surprise import KNNWithMeans, KNNBasic, SVD
from surprise import accuracy
from surprise.model_selection import train_test_split, cross_validate

import matplotlib.pyplot as plt
import numpy as np
import json
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances

In [37]:
class RefinedMyAlgo():
    def __init__(self, rating_data='', data_frame='', movie_data=''):
        if rating_data:
            reader = Reader(line_format='user item rating timestamp', sep=',')
            self.ratings = Dataset.load_from_file(rating_data, reader)
#             self.trainset, self.testset = train_test_split(self.ratings, test_size=0.25)
            self.trainset = self.ratings.build_full_trainset()
            self.sim_options = {'name': 'cosine','user_based': False}
            self.df_ratings = pd.read_csv(rating_data, low_memory=False, names=['userId', 'movieId', 'rating','timestamp'])
        elif not data_frame.empty:
            reader = Reader(rating_scale=(0, 5))
            self.ratings = Dataset.load_from_df(data_frame[['userId', 'movieId', 'rating']], reader)
            self.trainset = self.ratings.build_full_trainset()
            self.sim_options = {'name': 'cosine','user_based': False}
            
        if movie_data:
            self.movies = pd.read_csv(movie_data, low_memory=False)
            self.movies['year'] = self.movies['title'].apply(lambda x: x[-5:-1])
            self.movies['title'] = self.movies['title'].apply(lambda x: x[:-7])
            self.movies['genres'] = self.movies['genres'].apply(lambda x: x.replace('|',', '))
            
    
    def random_group(self, n):
        self.users_list = list(self.df_ratings['userId'])
        random_group = random.sample(self.users_list,n)
        return random_group
        

        
    def set_k(self, k_value=''):
        if k_value:
            algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
            self.algo = algo
            self.algo.fit(self.trainset)
        else:
            algo = SVD()
            self.algo = algo
            self.algo.fit(self.trainset)
        
        
    def find_best_k(self, k_value=''):
        if k_value:
            print('K = {}'.format(k_value))
            algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
            return cross_validate(algo, self.ratings, measures=['RMSE', 'MAE'], cv=10, verbose=True)
        else:
            aux = []
            for k_value in [3, 5, 7, 10, 15, 20, 30, 40]:
                print('K = {}'.format(k_value))
                algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
                my_dict = cross_validate(algo, self.ratings, measures=['RMSE', 'MAE'], cv=10, verbose=False)
                my_dict['k_value'] = k_value
                aux.append(my_dict)
            return aux
    
    
    def set_testset(self, users):
        if users:
            user_ratings = self.trainset.ur
            movies_ids = list(self.movies['movieId'])
            global_mean=self.trainset.global_mean
            my_testset = []
            
            for user in users:
                iuid = self.trainset.to_inner_uid(str(user))
                for movie in movies_ids:
                    is_in = False
                    for rating in user_ratings[iuid]:
#                         print( 'MOVIE: {}, RATING: {}'.format(movie,bla.trainset.to_raw_iid(rating[0])) )
                        if int(movie) == int(self.trainset.to_raw_iid(int(rating[0]))):
                            is_in = True
                            break
                    if not is_in:
                        my_tuple = (str(user),str(movie),global_mean)
                        my_testset.append(my_tuple)
                        
            self.testset = my_testset
        else:
            testset = self.trainset.build_anti_testset()
            self.testset = testset
        return self.testset


    def predict_ratings(self,users=''):
        # # Predict ratings for all pairs (u, i) that are NOT in the training set.
#         testset = self.trainset.build_anti_testset()
#         self.testset = testset
        testset = self.set_testset(users)
        predictions = self.algo.test(testset)
        self.predictions = predictions
        
        
    def set_perfil_movies(self, users):
        metadata = pd.read_csv('ml-latest-small/ratings.csv', low_memory=False, names=['userId', 'movieId', 'rating','timestamp'])
        metadata = metadata.drop(columns="timestamp")

        metadata_filtered = metadata[metadata.userId.isin(users)]

        self.group_sparse_mtx = pd.pivot_table(metadata_filtered, values='rating', index=['userId'], columns=['movieId'], fill_value=0)
        
        self.perfil_movies = list(self.group_sparse_mtx)
        
    
    ### You must call self.set_perfil_movies() before
    def set_candidate_movies(self):
        candidate_movies = []
        for item in refinedMyAlgo.movies.iterrows():
        #     get the movieId of each movie in movies dataframe
            if item[1].values[0] not in self.perfil_movies:
                candidate_movies.append(item[1].values[0])
        self.candidate_movies = candidate_movies
        
        
    def calc_similarity_matrix(self):
        #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
        tfidf = TfidfVectorizer(stop_words='english')
        
        #Replace NaN with an empty string
        self.movies['title'] = self.movies['title'].fillna('')
        self.movies['genres'] = self.movies['genres'].fillna('')
        
        #Construct the required TF-IDF matrix by fitting and transforming the data
        tfidf_matrix_title = tfidf.fit_transform(self.movies['title'])
        tfidf_matrix_genres = tfidf.fit_transform(self.movies['genres'])
        
        #Compute the cosine similarity matrix
        self.cosine_sim_movies_title = cosine_similarity(tfidf_matrix_title, tfidf_matrix_title)
        self.cosine_sim_movies_genres = cosine_similarity(tfidf_matrix_genres, tfidf_matrix_genres)
        
        
    def get_similar_movies(self, references, title_weight=0.8):
        recs = []
        for movie in references:
            # Get the pairwsie similarity scores of all movies with that movie
            movie_idx = int(self.movies[self.movies['movieId']==movie['movieID']].index[0])
            sim_scores_title = list(enumerate(self.cosine_sim_movies_title[movie_idx]))
            sim_scores_genres = list(enumerate(self.cosine_sim_movies_genres[movie_idx]))
            
            # Calculate total similarity based on title and genres
            total_sim_score = []
            for i in range(len(sim_scores_title)):
#                 print("sim_score_title= {}\t sim_score_genres= {}".format(sim_scores_title[i][1], sim_scores_genres[i][1]))
                aux = (sim_scores_title[i][1]*title_weight) + (sim_scores_genres[i][1]*(1-title_weight))
                total_sim_score.append((i, aux))
#                 print("sim_score_total= {}".format(total_sim_score))
                
            # Sort the movies based on the similarity scores
            total_sim_score = sorted(total_sim_score, key=lambda x: x[1], reverse=True)
            self.total_sim_score = total_sim_score
            
            candidates_sim_score = []
            for item in total_sim_score:
                if self.movies.loc[item[0]].values[0] not in self.perfil_movies:
                    candidates_sim_score.append(item)
            
            # Get the scores of the 10 most similar movies
            candidates_sim_score = candidates_sim_score[1:11]
            
            recs.append(candidates_sim_score)
            
        return recs
    
    
    def get_relevance_score(self, recs, references):
        count = 0
        recs_dict = []
        for reference in references:
        #     print('Referência: {}\t gêneros: {}'.format(refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==reference['movieID']].values[0][1], refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==reference['movieID']].values[0][2]))

            for movie in recs[count]:
                aux = {}

                movie_id = self.movies.loc[movie[0]].values[0]
                movie_title = self.movies.loc[movie[0]].values[1]
                movie_genres = self.movies.loc[movie[0]].values[2]
                movie_similarity = movie[1]
                movie_relevance = round(((reference['rating']/5.0)+movie_similarity)/2, 3)

                aux['movie_id'] = movie_id
                aux['movie_title'] = movie_title
                aux['movie_genres'] = movie_genres
                aux['movie_similarity'] = movie_similarity
                aux['movie_relevance'] = movie_relevance

                recs_dict.append(aux)

        #         print('\tSim: {},\trelevance: {},\tmovieId: {},\ttitle: {}'.format(aux['movie_similarity'], aux['movie_relevance'], aux['movie_id'], aux['movie_title']))

            count=count+1

        recs_dict = sorted(recs_dict, key = lambda i: i['movie_relevance'],reverse=True)

        return recs_dict
    
    
    def calc_distance_item_in_list(self, item, this_list, title_weight=0.8):

        idx_i = int(self.movies[self.movies['movieId']==item['movie_id']].index[0])

        total_dist = 0
        for movie in this_list:
            
            idx_j = int(self.movies[self.movies['movieId']==int(movie['movie_id'])].index[0])

            sim_i_j = (self.cosine_sim_movies_title[idx_i][idx_j]*title_weight) + (self.cosine_sim_movies_genres[idx_i][idx_j]*(1-title_weight))
            dist_i_j = 1 - sim_i_j
            total_dist = total_dist + dist_i_j

        result = total_dist/len(this_list)
        return result
    
    
    def calc_diversity_score(self, actual_list, candidates_list, alfa=0.5):
        '''
        This function implemented here was based on MARIUS KAMINSKAS and DEREK BRIDGE paper: Diversity, Serendipity, Novelty, and Coverage: A Survey and Empirical Analysis of Beyond-Accuracy Objectives in Recommender Systems
        func(i,R) = (relevance[i]*alfa) + (dist_i_R(i,R)*(1-alfa))
        '''
        diversity_score = []
        count = 0

        for item in candidates_list:

            aux = {}
            dist_item_R = self.calc_distance_item_in_list(item=item, this_list=actual_list)
            aux['div_score'] = (item['movie_relevance']*alfa) + (dist_item_R*(1-alfa))
            aux['idx'] = count
            diversity_score.append(aux)
            count = count + 1

        return diversity_score
    
    
    def diversify_recs_list(self, recs, n=10):
        '''
        This function implemented here was based on MARIUS KAMINSKAS and DEREK BRIDGE paper: Diversity, Serendipity, Novelty, and Coverage: A Survey and Empirical Analysis of Beyond-Accuracy Objectives in Recommender Systems
        The Greedy Reranking Algorithm.
        '''
        diversified_list = []
        
        while len(diversified_list) < n:
            if len(diversified_list) == 0:
                diversified_list.append(recs[0])
                recs.pop(0)
            else:
                diversity_score = self.calc_diversity_score(actual_list=diversified_list, candidates_list=recs)
                diversity_score = sorted(diversity_score, key = lambda i: i['div_score'],reverse=True)
#               #  Add the item that maximize diversity in the list 
                item = diversity_score[0]
                diversified_list.append(recs[item['idx']])
#               #  Remove this item from the candidates list
                recs.pop(item['idx'])
    
        return diversified_list
                
        
#     def get_ILS(self, final_recs, title_weight=0.8):
#         big_list = []
#         for i in final_recs:
#             movie_idx_i = int(self.movies[self.movies['movieId']==i['movie_id']].index[0])
#             small_list = []
#             for j in final_recs:
#                 movie_idx_j = int(self.movies[self.movies['movieId']==j['movie_id']].index[0])
#                 sim_genre = self.cosine_sim_movies_genres[movie_idx_i][movie_idx_j]
#                 sim_title = self.cosine_sim_movies_title[movie_idx_i][movie_idx_j]
#                 total_sim = (sim_title*title_weight) + (sim_genre*(1-title_weight))
#                 small_list.append(total_sim)
#             big_list.append(small_list)

#         return big_list

    def calc_dist_i_j(self, idx_i, idx_j, title_weight=0.8):
        sim_genre = self.cosine_sim_movies_genres[movie_idx_i][movie_idx_j]
        sim_title = self.cosine_sim_movies_title[movie_idx_i][movie_idx_j]
        total_sim = (sim_title*title_weight) + (sim_genre*(1-title_weight))
        dist_score = 1 - total_sim

        return dist_score
    
    
    def get_distance_matrix(self, final_recs, title_weight=0.8):
        dist_matrix = []
        for i in final_recs:
            aux = []
            movie_idx_i = int(self.movies[self.movies['movieId']==i['movie_id']].index[0])
            for j in final_recs:
                movie_idx_j = int(self.movies[self.movies['movieId']==j['movie_id']].index[0])
                dist_i_j = self.calc_dist_i_j(movie_idx_i, movie_idx_j, title_weight=0.8)
                aux.append(dist_i_j)
            dist_matrix.append(aux)
            
        return dist_matrix
    
    def get_ILD_score(self, final_recs, title_weight=0.8):
        dist_matrix = self.get_distance_matrix(final_recs, title_weight=0.8)
        np_dist_mtx = np.array(dist_matrix)
        upper_right = np.triu_indices(np_dist_mtx.shape[0], k=1)

        ild_score = np.mean(np_mtx[upper_right])
        
        return ild_score

In [38]:
refinedMyAlgo = RefinedMyAlgo(rating_data='ml-latest-small/ratings.csv', movie_data='ml-latest-small/movies.csv')
refinedMyAlgo.set_k()

In [49]:
# print(refinedMyAlgo.random_group(3))

In [4]:
# # # FIXED GROUP
# my_group = [77,596,452,243,420]

# # # RANDOM GROUP
my_group = refinedMyAlgo.random_group(5)

refinedMyAlgo.predict_ratings(users=my_group)
len(refinedMyAlgo.predictions)

47891

In [5]:
# metadata = pd.read_csv('ml-latest-small/ratings.csv', low_memory=False, names=['userId', 'movieId', 'rating','timestamp'])
# metadata = metadata.drop(columns="timestamp")

# metadata_filtered = metadata[metadata.userId.isin(my_group)]

# my_group_sparse = pd.pivot_table(metadata_filtered, values='rating', index=['userId'], columns=['movieId'], fill_value=0)
# my_group_sparse.head()

refinedMyAlgo.set_perfil_movies(users=my_group)
refinedMyAlgo.set_candidate_movies()

# print(refinedMyAlgo.perfil_movies)
# print(refinedMyAlgo.candidate_movies)
refinedMyAlgo.group_sparse_mtx.head()

movieId,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
userId,,,,,,,,,,,,,,,,,,,,,
77,0,0,0.0,0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
243,0,5,0.0,0,4,0,4,0,4,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
420,4,0,3.5,0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
452,0,4,0.0,0,0,0,4,5,0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
596,4,0,3.5,4,0,4,0,0,0,3.5,...,2.5,3.5,3.5,4,3.5,3.5,3.5,3.5,4,4


In [6]:
group_filled_mtx = refinedMyAlgo.group_sparse_mtx.copy()

for index, row in group_filled_mtx.iterrows():
    for col in list(group_filled_mtx):
        if(group_filled_mtx.loc[index,col] == 0.0):
            aux = list(filter(lambda x: x.uid==str(index) and x.iid==str(col), refinedMyAlgo.predictions))
            group_filled_mtx.loc[index,col] = aux[0].est

group_filled_mtx = group_filled_mtx.round(decimals=3)
group_filled_mtx.head()

movieId,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
userId,,,,,,,,,,,,,,,,,,,,,
77,3.722,3.799,3.922,3.620,3.609,3.655,2.816,4.176,3.165,4.135,...,3.326,3.412,3.653,3.695,3.161,3.591,3.814,3.493,3.588,3.551
243,4.699,5.000,4.776,4.502,4.000,4.241,4.000,4.672,4.000,5.000,...,4.015,4.307,4.210,4.366,4.087,4.163,4.320,4.185,4.256,4.266
420,4.000,3.527,3.500,3.831,4.168,3.449,2.655,4.156,3.357,4.297,...,3.296,3.417,3.597,3.626,3.243,3.713,3.461,3.463,3.681,3.734
452,4.676,4.000,4.553,4.305,4.626,4.629,4.000,5.000,4.117,5.000,...,4.228,4.325,4.590,4.518,4.138,4.293,4.469,4.365,4.747,4.392
596,4.000,3.300,3.500,4.000,3.530,4.000,2.890,3.741,3.020,3.500,...,2.500,3.500,3.500,4.000,3.500,3.500,3.500,3.500,4.000,4.000


In [7]:
########################################################################
# # Implementing LEAST MISERY ending-up in a dataframe
########################################################################
values = []
labels = []
for i in range(0,len(list(group_filled_mtx))):
    my_col = group_filled_mtx.iloc[ : ,i]
    label = my_col.name
    my_col = list(my_col)
    
    labels.append(label)
    values.append( float(min(my_col)) )
    
# print('Array values: {}, Array labels: {}'.format(values, labels))
agg_group_perf = pd.DataFrame(index=[900], columns=labels)

for i in range(0,len(list(agg_group_perf))):
    agg_group_perf.iloc[0, i] = values[i]

    
agg_group_perf = agg_group_perf.round(decimals=3)
agg_group_perf.head()

,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
900,3.745,3.344,3.5,3.652,3.647,3.422,2.706,3.997,3.123,3.5,...,2.5,3.251,3.5,3.701,3.173,3.5,3.339,3.5,3.358,3.579


In [ ]:
########################################################################
# # Implementing MOST PLEASURE ending-up in a dataframe
########################################################################
values = []
labels = []
for i in range(0,len(list(group_filled_mtx))):
    my_col = group_filled_mtx.iloc[ : ,i]
    label = my_col.name
    my_col = list(my_col)
    
    labels.append(label)
    values.append( float(max(my_col)) )
    
# print('Array values: {}, Array labels: {}'.format(values, labels))
agg_group_perf = pd.DataFrame(index=[900], columns=labels)

for i in range(0,len(list(agg_group_perf))):
    agg_group_perf.iloc[0, i] = values[i]

    
agg_group_perf = agg_group_perf.round(decimals=3)
agg_group_perf.head()

In [7]:
########################################################################
# # Implementing AVERAGE WITHOUT MISERY: treshold=2  ending-up in a dataframe
########################################################################
values = []
labels = []
for i in range(0,len(list(group_filled_mtx))):
    my_col = group_filled_mtx.iloc[ : ,i]
    label = my_col.name
    my_col = list(my_col)
    
    labels.append(label)
    if float(min(my_col)) <= 2 :
        values.append( float(min(my_col)) )
    else:
        values.append( float( sum(my_col) / len(my_col) ) )
    
# print('Array values: {}, Array labels: {}'.format(values, labels))
agg_group_perf = pd.DataFrame(index=[900], columns=labels)

for i in range(0,len(list(agg_group_perf))):
    agg_group_perf.iloc[0, i] = values[i]

    
agg_group_perf = agg_group_perf.round(decimals=3)
agg_group_perf.head()

,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
900,4.2194,3.9252,4.0502,4.0516,3.9866,3.9948,3.2722,4.349,3.5318,4.3864,...,3.473,3.7922,3.91,4.041,3.6258,3.852,3.9128,3.8012,4.0544,3.9886


In [8]:
group_pref_dict = []
for col in list(agg_group_perf):
    my_dict = {}
#     print('Valor: {}, Coluna: {}'.format(agg_group_perf.loc[900,col], col))
    my_dict['rating'] = agg_group_perf.loc[900,col]
    my_dict['movieID'] = col
    group_pref_dict.append(my_dict)
    
group_pref_dict = sorted(group_pref_dict, key = lambda i: i['rating'],reverse=True)
group_pref_dict

[{'rating': 4.6498, 'movieID': 260},
 {'rating': 4.647, 'movieID': 57669},
 {'rating': 4.6454, 'movieID': 4993},
 {'rating': 4.6452, 'movieID': 168252},
 {'rating': 4.6442, 'movieID': 1136},
 {'rating': 4.643, 'movieID': 2571},
 {'rating': 4.6292, 'movieID': 58559},
 {'rating': 4.622, 'movieID': 904},
 {'rating': 4.617, 'movieID': 38061},
 {'rating': 4.5944, 'movieID': 318},
 {'rating': 4.587999999999999, 'movieID': 5971},
 {'rating': 4.57, 'movieID': 3275},
 {'rating': 4.563599999999999, 'movieID': 7153},
 {'rating': 4.5536, 'movieID': 3000},
 {'rating': 4.5476, 'movieID': 51255},
 {'rating': 4.5424, 'movieID': 1198},
 {'rating': 4.534000000000001, 'movieID': 3996},
 {'rating': 4.519799999999999, 'movieID': 31658},
 {'rating': 4.5162, 'movieID': 750},
 {'rating': 4.516, 'movieID': 1196},
 {'rating': 4.513, 'movieID': 60069},
 {'rating': 4.5118, 'movieID': 1222},
 {'rating': 4.5036000000000005, 'movieID': 5618},
 {'rating': 4.5016, 'movieID': 5952},
 {'rating': 4.5008, 'movieID': 44555

In [9]:
refinedMyAlgo.calc_similarity_matrix()

In [10]:
references = group_pref_dict[0:10]
# references = group_pref_dict

for item in references:
    print(item)
# print(references)

{'rating': 4.6498, 'movieID': 260}
{'rating': 4.647, 'movieID': 57669}
{'rating': 4.6454, 'movieID': 4993}
{'rating': 4.6452, 'movieID': 168252}
{'rating': 4.6442, 'movieID': 1136}
{'rating': 4.643, 'movieID': 2571}
{'rating': 4.6292, 'movieID': 58559}
{'rating': 4.622, 'movieID': 904}
{'rating': 4.617, 'movieID': 38061}
{'rating': 4.5944, 'movieID': 318}


In [11]:
# references=[904, 318, 38061, 31658, 57669]
recs = refinedMyAlgo.get_similar_movies(references)

In [12]:
# print(refinedMyAlgo.total_sim_score)

In [13]:
# amarelo = references
# vermelho = recs

# count = 0
# recs_dict = []
# for reference in amarelo:
# #     print('Referência: {}\t gêneros: {}'.format(refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==reference['movieID']].values[0][1], refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==reference['movieID']].values[0][2]))

#     for movie in vermelho[count]:
#         aux = {}

#         movie_id = refinedMyAlgo.movies.loc[movie[0]].values[0]
#         movie_title = refinedMyAlgo.movies.loc[movie[0]].values[1]
#         movie_genres = refinedMyAlgo.movies.loc[movie[0]].values[2]
#         movie_similarity = movie[1]
#         movie_relevance = ((reference['rating']/5.0)+movie_similarity)/2

#         aux['movie_id'] = movie_id
#         aux['movie_title'] = movie_title
#         aux['movie_genres'] = movie_genres
#         aux['movie_similarity'] = movie_similarity
#         aux['movie_relevance'] = movie_relevance

#         recs_dict.append(aux)

#         print('\tSim: {},\trelevance: {},\tmovieId: {},\ttitle: {}'.format(aux['movie_similarity'], aux['movie_relevance'], aux['movie_id'], aux['movie_title']))

#     count=count+1

# recs_dict = sorted(recs_dict, key = lambda i: i['movie_relevance'],reverse=True)

# print("@ @ @ @ @ @ @ @ @ @ @ @ @")
# recs_dict

In [12]:
candidates_list = refinedMyAlgo.get_relevance_score(recs=recs, references=references)
print(len(candidates_list))

for item in candidates_list[0:20]:
    print('relevance: {}, title:{}'.format(item['movie_relevance'], item['movie_title']))

100
relevance: 0.763, title:Batman: The Dark Knight Returns, Part 2
relevance: 0.759, title:Logan Lucky
relevance: 0.744, title:Cry in the Dark, A
relevance: 0.744, title:Last Kiss, The
relevance: 0.742, title:Big Bang, The
relevance: 0.733, title:Alone in the Dark
relevance: 0.731, title:Knight and Day
relevance: 0.72, title:Kiss of Death
relevance: 0.706, title:Undisputed III: Redemption
relevance: 0.702, title:Black Knight
relevance: 0.7, title:Dark Blue
relevance: 0.695, title:Solo: A Star Wars Story
relevance: 0.694, title:Redemption (Hummingbird)
relevance: 0.691, title:Comic-Con Episode IV: A Fan's Hope
relevance: 0.683, title:Dark Shadows
relevance: 0.68, title:Alone in the Dark II
relevance: 0.677, title:The Star Wars Holiday Special
relevance: 0.677, title:Batman: Gotham Knight
relevance: 0.675, title:Chitty Chitty Bang Bang
relevance: 0.672, title:Holy Man


In [13]:
dist_i_R = refinedMyAlgo.calc_distance_item_in_list(item=candidates_list[24],this_list=candidates_list[0:5])
dist_i_R

0.8634217075818047

In [14]:
print(refinedMyAlgo.calc_diversity_score(actual_list=candidates_list[0:5], candidates_list=candidates_list[10:20]))

[{'div_score': 0.6400003439754277, 'idx': 0}, {'div_score': 0.7894240238852528, 'idx': 1}, {'div_score': 0.7484724100424557, 'idx': 2}, {'div_score': 0.8367458299867431, 'idx': 3}, {'div_score': 0.8454999999999999, 'idx': 4}, {'div_score': 0.7623896857546567, 'idx': 5}, {'div_score': 0.7449984560299496, 'idx': 6}, {'div_score': 0.7851547503872609, 'idx': 7}, {'div_score': 0.7404984560299497, 'idx': 8}, {'div_score': 0.7404984560299497, 'idx': 9}]


In [13]:
my_candidates = candidates_list.copy()
final_recs = refinedMyAlgo.diversify_recs_list(recs=my_candidates)
for item in final_recs:
    print('relevance: {}, title:{}'.format(item['movie_relevance'], item['movie_title']))

relevance: 0.763, title:Batman: The Dark Knight Returns, Part 2
relevance: 0.759, title:Logan Lucky
relevance: 0.72, title:Kiss of Death
relevance: 0.691, title:Comic-Con Episode IV: A Fan's Hope
relevance: 0.742, title:Big Bang, The
relevance: 0.695, title:Solo: A Star Wars Story
relevance: 0.744, title:Cry in the Dark, A
relevance: 0.706, title:Undisputed III: Redemption
relevance: 0.672, title:Holy Man
relevance: 0.656, title:Ring, The


In [16]:
refinedMyAlgo.get_ILD_score(final_recs, title_weight=0.8)

0.9683581044592141